<a href="https://colab.research.google.com/github/Amisha2323/adamdoyle/blob/main/midterm/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [27]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [28]:
def calculate_bmi(height_m, weight_kg):
    """
    Calculate BMI given height in meters and weight in kilograms.

    Args:
    height_m (float): Height in meters
    weight_kg (float): Weight in kilograms

    Returns:
    float: BMI value

    >>> calculate_bmi(1.67, 72)
    22.74
    >>> calculate_bmi(1.5, 58)
    22.36
    >>> calculate_bmi(0, 60)
    Traceback (most recent call last):
    ...
    ValueError: Height and weight must be positive numbers
    """
    if height_m <= 0 or weight_kg <= 0:
        raise ValueError("Height and weight must be positive numbers")
    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

# Run doctests
doctest.testmod()

**********************************************************************
File "__main__", line 12, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.67, 72)
Expected:
    22.74
Got:
    25.82
**********************************************************************
File "__main__", line 14, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.5, 58)
Expected:
    22.36
Got:
    25.78
**********************************************************************
File "__main__", line 13, in __main__.calculate_bode_risk
Failed example:
    calculate_bode_risk(5)
Expected:
    0.6
Got:
    0.57
**********************************************************************
File "__main__", line 14, in __main__.calculate_bode_score
Failed example:
    calculate_bode_score(65, 350, "STOPS AFTER 100 YARDS", 22)
Expected:
    3
Got:
    2
**********************************************************************
File "__main__", line 16, in __main__.calculate_bode_score
Failed example:
    calcula

TestResults(failed=5, attempted=9)

### Step 2: Calculate BODE Score

In [29]:
def calculate_bode_score(fev_pct, distance_meters, dyspnea_description, bmi):
    """
    Calculate BODE score based on given parameters.

    Args:
    fev_pct (float): FEV1 % predicted
    distance_meters (float): 6-minute walk distance in meters
    dyspnea_description (str): Description of dyspnea
    bmi (float): Body Mass Index

    Returns:
    int: BODE score (0-10)

    >>> calculate_bode_score(65, 350, "STOPS AFTER 100 YARDS", 22)
    3
    >>> calculate_bode_score(35, 150, "BREATHLESS WHEN DRESSING", 19)
    7
    >>> calculate_bode_score(80, 400, "ONLY STRENUOUS EXERCISE", 25)
    0
    """
    score = 0

    # FEV1 score
    if fev_pct >= 65:
        score += 0
    elif fev_pct >= 50:
        score += 1
    elif fev_pct >= 36:
        score += 2
    else:
        score += 3

    # 6-minute walk distance score
    if distance_meters >= 350:
        score += 0
    elif distance_meters >= 250:
        score += 1
    elif distance_meters >= 150:
        score += 2
    else:
        score += 3

    # MMRC Dyspnea score
    dyspnea_levels = {
        "ONLY STRENUOUS EXERCISE": 0,
        "WHEN HURRYING": 1,
        "WALKING UPHILL": 1,
        "SLOWER THAN PEERS": 2,
        "STOPS AFTER 100 YARDS": 2,
        "STOPS AFTER A FEW MINUTES": 3,
        "BREATHLESS WHEN DRESSING": 3,
        "UNABLE TO LEAVE HOME": 4
    }
    score += dyspnea_levels.get(dyspnea_description.upper(), 0)

    # BMI score
    if bmi > 21:
        score += 0
    else:
        score += 1

    return score

# Run doctests
doctest.testmod()

**********************************************************************
File "__main__", line 12, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.67, 72)
Expected:
    22.74
Got:
    25.82
**********************************************************************
File "__main__", line 14, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.5, 58)
Expected:
    22.36
Got:
    25.78
**********************************************************************
File "__main__", line 13, in __main__.calculate_bode_risk
Failed example:
    calculate_bode_risk(5)
Expected:
    0.6
Got:
    0.57
**********************************************************************
File "__main__", line 14, in __main__.calculate_bode_score
Failed example:
    calculate_bode_score(65, 350, "STOPS AFTER 100 YARDS", 22)
Expected:
    3
Got:
    2
**********************************************************************
File "__main__", line 16, in __main__.calculate_bode_score
Failed example:
    calcula

TestResults(failed=5, attempted=9)

### Step 3: Calculate BODE Risk

In [30]:
def calculate_bode_risk(bode_score):
    """
    Calculate 4-year survival rate based on BODE score.

    Args:
    bode_score (int): BODE score (0-10)

    Returns:
    float: 4-year survival rate as a percentage

    >>> calculate_bode_risk(0)
    0.8
    >>> calculate_bode_risk(5)
    0.6
    >>> calculate_bode_risk(10)
    0.2
    """
    if bode_score <= 2:
        return 0.8
    elif bode_score <= 4:
        return 0.67
    elif bode_score <= 6:
        return 0.57
    elif bode_score <= 7:
        return 0.45
    else:
        return 0.2

# Run doctests
doctest.testmod()

**********************************************************************
File "__main__", line 12, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.67, 72)
Expected:
    22.74
Got:
    25.82
**********************************************************************
File "__main__", line 14, in __main__.calculate_bmi
Failed example:
    calculate_bmi(1.5, 58)
Expected:
    22.36
Got:
    25.78
**********************************************************************
File "__main__", line 13, in __main__.calculate_bode_risk
Failed example:
    calculate_bode_risk(5)
Expected:
    0.6
Got:
    0.57
**********************************************************************
File "__main__", line 14, in __main__.calculate_bode_score
Failed example:
    calculate_bode_score(65, 350, "STOPS AFTER 100 YARDS", 22)
Expected:
    3
Got:
    2
**********************************************************************
File "__main__", line 16, in __main__.calculate_bode_score
Failed example:
    calcula

TestResults(failed=5, attempted=9)

### Step 4: Load Hospital Data

In [31]:
def load_hospital_data(json_file):
    """
    Load hospital data from a JSON file.

    Args:
    json_file (str): Path to the JSON file

    Returns:
    dict: Hospital data with hospital names as keys and bed counts as values
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    hospital_beds = {}
    for system in data:
        for hospital in system['hospitals']:
            hospital_beds[hospital['name']] = hospital['beds']

    return hospital_beds

### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [32]:
import csv

def process_patient_data(patient_csv, hospital_json, patient_output_file, hospital_output_file):
    # Load hospital data
    hospital_beds = load_hospital_data(hospital_json)

    # Initialize variables for hospital statistics
    hospital_stats = {hospital: {'count': 0, 'total_score': 0, 'total_risk': 0} for hospital in hospital_beds}

    # Process patient data
    patient_results = []
    with open(patient_csv, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row['NAME']
            height_m = float(row['HEIGHT_M'])
            weight_kg = float(row['WEIGHT_KG'])
            fev_pct = float(row['fev_pct'])
            dyspnea_description = row['dyspnea_description']
            distance_meters = float(row['distance_in_meters'])
            hospital = row['hospital']

            bmi = calculate_bmi(height_m, weight_kg)
            bode_score = calculate_bode_score(fev_pct, distance_meters, dyspnea_description, bmi)
            bode_risk = calculate_bode_risk(bode_score)

            patient_results.append([name, bode_score, bode_risk, hospital])

            # Update hospital statistics
            hospital_stats[hospital]['count'] += 1
            hospital_stats[hospital]['total_score'] += bode_score
            hospital_stats[hospital]['total_risk'] += bode_risk

    # Write patient output file
    with open(patient_output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['NAME', 'BODE_SCORE', 'BODE_RISK', 'HOSPITAL'])
        writer.writerows(patient_results)

    # Calculate and write hospital output
    hospital_output = []
    for hospital, stats in hospital_stats.items():
        copd_count = stats['count']
        if copd_count > 0:
            avg_score = stats['total_score'] / copd_count
            avg_risk = stats['total_risk'] / copd_count
            pct_cases_over_beds = (copd_count / hospital_beds[hospital]) * 100
            hospital_output.append([
                hospital,
                copd_count,
                round(pct_cases_over_beds, 2),
                round(avg_score, 2),
                round(avg_risk, 2)
            ])

    with open(hospital_output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['HOSPITAL_NAME', 'COPD_COUNT', 'PCT_OF_COPD_CASES_OVER_BEDS', 'AVG_SCORE', 'AVG_RISK'])
        writer.writerows(hospital_output)

# Process the data
process_patient_data('patient.csv', 'hospitals.json', 'patient_output.csv', 'hospital_output.csv')